In [5]:
#!/usr/bin/env python3
"""
Anomalous Federated Learning Client (Non-Poisoning)
===================================================

This client creates various types of anomalies without traditional model poisoning:
- Parameter distribution anomalies
- Training behavior anomalies  
- Performance inconsistencies
- Statistical outliers
- Pattern irregularities

These anomalies test the anomaly detection system specifically.
"""

import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging
import random
from typing import Dict, List, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("AnomalousClient")

# Client configuration
ANOMALY_CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01,
    
    # Anomaly configuration
    "anomaly_type": "mixed",  # "parameter", "training", "performance", "statistical", "mixed"
    "anomaly_intensity": 0.7,  # 0.0 (none) to 1.0 (maximum)
    "anomaly_frequency": 0.8,  # Probability of introducing anomaly each round
}

class HeartDiseaseModel(nn.Module):
    """Heart disease prediction model."""
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(ANOMALY_CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(ANOMALY_CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

class FedProxLoss(nn.Module):
    """FedProx loss function."""
    def __init__(self, base_criterion, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.base_criterion = base_criterion
        self.mu = mu
        
    def forward(self, y_pred, y_true, model_params, global_params):
        base_loss = self.base_criterion(y_pred, y_true)
        
        proximal_term = 0.0
        if global_params is not None:
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
            loss = base_loss + (self.mu / 2) * proximal_term
            return loss
        
        return base_loss

class AnomalyGenerator:
    """Generates various types of anomalies for testing."""
    
    def __init__(self, anomaly_type="mixed", intensity=0.7, frequency=0.8):
        self.anomaly_type = anomaly_type
        self.intensity = intensity
        self.frequency = frequency
        self.round_count = 0
        
    def should_introduce_anomaly(self):
        """Decide whether to introduce anomaly this round."""
        return random.random() < self.frequency
    
    def apply_parameter_anomalies(self, parameters: List[np.ndarray]) -> List[np.ndarray]:
        """Apply parameter-level anomalies (non-poisoning)."""
        anomalous_params = []
        
        for i, param in enumerate(parameters):
            anomalous_param = param.copy()
            
            if self.anomaly_type in ["parameter", "mixed"]:
                # Type 1: Add structured noise (more aggressive)
                if random.random() < 0.7:
                    noise_scale = 0.3 * self.intensity  # Increased from 0.1
                    structured_noise = np.sin(np.arange(param.size) * 0.1).reshape(param.shape) * noise_scale
                    anomalous_param += structured_noise
                    logger.info(f"  Applied STRONG structured noise to layer {i} (scale={noise_scale:.3f})")
                
                # Type 2: Introduce more aggressive sparsity patterns
                if random.random() < 0.5:
                    sparsity_ratio = 0.4 * self.intensity  # Increased from 0.2
                    mask = np.random.random(param.shape) < sparsity_ratio
                    anomalous_param[mask] = 0
                    logger.info(f"  Applied AGGRESSIVE sparsity to layer {i} ({np.sum(mask)} zeros, {sparsity_ratio:.1%} sparse)")
                
                # Type 3: Amplify parameters more dramatically
                if random.random() < 0.6 and param.size > 10:
                    amplification = 1 + 1.5 * self.intensity  # Increased from 0.5
                    # Amplify every 2nd parameter instead of every 3rd
                    indices = np.arange(0, param.size, 2)
                    flat_param = anomalous_param.flatten()
                    flat_param[indices] *= amplification
                    anomalous_param = flat_param.reshape(param.shape)
                    logger.info(f"  Applied STRONG amplification to layer {i} (factor={amplification:.2f})")
                
                # Type 4: Add systematic bias (new)
                if random.random() < 0.4:
                    bias = 0.5 * self.intensity * np.sign(np.mean(param))
                    anomalous_param += bias
                    logger.info(f"  Added systematic bias to layer {i} (bias={bias:.3f})")
                
                # Type 5: Create artificial patterns (new)
                if random.random() < 0.3 and param.size > 50:
                    pattern_size = min(10, param.size // 10)
                    pattern = np.tile([1, -1], pattern_size // 2 + 1)[:pattern_size]
                    pattern = pattern * 0.3 * self.intensity
                    flat_param = anomalous_param.flatten()
                    flat_param[:pattern_size] = pattern
                    anomalous_param = flat_param.reshape(param.shape)
                    logger.info(f"  Applied artificial pattern to layer {i}")
            
            anomalous_params.append(anomalous_param)
        
        return anomalous_params
    
    def get_anomalous_training_config(self) -> Dict:
        """Get anomalous training configuration."""
        config = {}
        
        if self.anomaly_type in ["training", "mixed"]:
            # Anomalous learning rates
            if random.random() < 0.4:
                if random.random() < 0.5:
                    config["learning_rate"] = 0.1 * self.intensity  # Too high
                    logger.info(f"  Using anomalously high learning rate: {config['learning_rate']}")
                else:
                    config["learning_rate"] = 0.0001 * (1 - self.intensity)  # Too low
                    logger.info(f"  Using anomalously low learning rate: {config['learning_rate']}")
            
            # Anomalous batch processing
            if random.random() < 0.3:
                config["process_partial_batches"] = True
                config["partial_ratio"] = 0.3 + 0.4 * self.intensity
                logger.info(f"  Processing only {config['partial_ratio']:.2f} of each batch")
            
            # Anomalous epoch patterns
            if random.random() < 0.2:
                config["skip_epochs"] = int(1 + 2 * self.intensity)
                logger.info(f"  Skipping {config['skip_epochs']} training epochs")
        
        return config
    
    def get_anomalous_metrics(self, true_accuracy: float, true_loss: float) -> Dict:
        """Generate anomalous (but not obviously fake) metrics."""
        metrics = {
            "accuracy": true_accuracy,
            "loss": true_loss,
            "poisoning_active": False  # Not doing traditional poisoning
        }
        
        if self.anomaly_type in ["performance", "mixed"]:
            # Type 1: Extreme inconsistent performance (more aggressive)
            if random.random() < 0.6:
                # Very high accuracy but very high loss (clearly inconsistent)
                metrics["accuracy"] = min(0.95, true_accuracy + 0.4 * self.intensity)
                metrics["loss"] = max(true_loss, true_loss + 2.0 * self.intensity)
                logger.info(f"  Reporting EXTREME inconsistent metrics: acc={metrics['accuracy']:.3f}, loss={metrics['loss']:.3f}")
            
            # Type 2: Oscillating performance (more extreme)
            elif random.random() < 0.4:
                oscillation = 0.5 * self.intensity * np.sin(self.round_count * np.pi / 2)
                metrics["accuracy"] = max(0.05, min(0.95, true_accuracy + oscillation))
                metrics["loss"] = max(0.1, true_loss - oscillation * 0.5)
                logger.info(f"  Applying EXTREME performance oscillation: {oscillation:.3f}")
            
            # Type 3: Suspicious performance patterns
            elif random.random() < 0.3:
                # Performance that's too good for early rounds
                if self.round_count <= 3:
                    metrics["accuracy"] = 0.85 + 0.1 * self.intensity
                    metrics["loss"] = 0.2 + 0.1 * self.intensity
                    logger.info(f"  Reporting suspiciously good early performance")
                
                # Or performance that degrades suddenly
                else:
                    metrics["accuracy"] = max(0.1, true_accuracy - 0.3 * self.intensity)
                    metrics["loss"] = true_loss + 1.5 * self.intensity
                    logger.info(f"  Reporting sudden performance degradation")
        
        return metrics

def load_data(data_path):
    """Load and preprocess data."""
    try:
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        if df.isnull().sum().sum() > 0:
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=ANOMALY_CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

class AnomalousFraminghamClient(fl.client.NumPyClient):
    """Anomalous client that generates various anomalies for testing."""
    
    def __init__(self, model, dataloader, device, client_id, anomaly_config=None):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.client_id = client_id
        self.global_params = None
        
        # Initialize anomaly generator
        if anomaly_config is None:
            anomaly_config = {
                "anomaly_type": ANOMALY_CLIENT_CONFIG["anomaly_type"],
                "intensity": ANOMALY_CLIENT_CONFIG["anomaly_intensity"],
                "frequency": ANOMALY_CLIENT_CONFIG["anomaly_frequency"]
            }
        
        self.anomaly_generator = AnomalyGenerator(**anomaly_config)
        
        logger.info(f"[ANOMALOUS] Client {client_id} initialized")
        logger.info(f"  Anomaly Type: {anomaly_config['anomaly_type']}")
        logger.info(f"  Intensity: {anomaly_config['intensity']}")
        logger.info(f"  Frequency: {anomaly_config['frequency']}")
        
    def get_parameters(self, config):
        """Get model parameters with potential anomalies."""
        params = [val.detach().cpu().numpy() for val in self.model.parameters()]
        
        # Apply anomalies if decided for this round
        if self.anomaly_generator.should_introduce_anomaly():
            logger.info(f"[ANOMALY] Client {self.client_id} introducing anomalies...")
            params = self.anomaly_generator.apply_parameter_anomalies(params)
            
            # Log parameter statistics
            original_norm = sum(np.linalg.norm(p) for p in [val.detach().cpu().numpy() for val in self.model.parameters()])
            anomalous_norm = sum(np.linalg.norm(p) for p in params)
            logger.info(f"  Parameter norm: {original_norm:.6f} -> {anomalous_norm:.6f}")
        else:
            logger.info(f"[NORMAL] Client {self.client_id} behaving normally this round")
        
        return params
    
    def set_parameters(self, parameters):
        """Set model parameters."""
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info(f"Client {self.client_id}: parameters updated from server")
    
    def fit(self, parameters, config):
        """Train with potential anomalies."""
        self.set_parameters(parameters)
        self.anomaly_generator.round_count += 1
        
        # Get anomalous training configuration
        anomaly_config = self.anomaly_generator.get_anomalous_training_config()
        
        # Apply anomalous training settings
        learning_rate = anomaly_config.get("learning_rate", ANOMALY_CLIENT_CONFIG["learning_rate"])
        process_partial = anomaly_config.get("process_partial_batches", False)
        partial_ratio = anomaly_config.get("partial_ratio", 1.0)
        skip_epochs = anomaly_config.get("skip_epochs", 0)
        
        if anomaly_config:
            logger.info(f"[ANOMALY] Client {self.client_id} using anomalous training config")
        
        self.model.train()
        criterion = nn.BCELoss()
        proximal_criterion = FedProxLoss(criterion, mu=ANOMALY_CLIENT_CONFIG["proximal_mu"])
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=learning_rate,
            weight_decay=ANOMALY_CLIENT_CONFIG["weight_decay"]
        )
        
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Training with potential anomalies
        effective_epochs = max(1, ANOMALY_CLIENT_CONFIG["local_epochs"] - skip_epochs)
        
        for epoch in range(effective_epochs):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Anomalous batch processing
                if process_partial and random.random() > partial_ratio:
                    continue  # Skip this batch
                
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                
                # Loss calculation
                loss = proximal_criterion(
                    y_pred, y, 
                    self.model.parameters(),
                    self.global_params
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                if batch_idx % 10 == 0:
                    logger.info(f"Client {self.client_id} - Epoch {epoch+1}/{effective_epochs} - Batch {batch_idx} - Loss: {loss.item():.4f}")
        
        # Calculate metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        # Apply anomalous metric reporting
        anomalous_metrics = self.anomaly_generator.get_anomalous_metrics(accuracy, avg_loss)
        
        logger.info(f"[TRAINING] Client {self.client_id} training completed")
        logger.info(f"  True metrics: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")
        logger.info(f"  Reported metrics: Loss={anomalous_metrics['loss']:.4f}, Accuracy={anomalous_metrics['accuracy']:.4f}")
        
        return self.get_parameters({}), total_samples, {
            "loss": float(anomalous_metrics["loss"]), 
            "accuracy": float(anomalous_metrics["accuracy"]),
            "client_id": str(self.client_id),
            "poisoning_active": False,  # Not traditional poisoning
            "anomaly_active": True,     # But anomalous behavior
            "anomaly_type": self.anomaly_generator.anomaly_type,
            "anomaly_intensity": self.anomaly_generator.intensity
        }
    
    def evaluate(self, parameters, config):
        """Evaluate with potential anomalies."""
        self.set_parameters(parameters)
        
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                X, y = X.to(self.device), y.to(self.device)
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        # Apply anomalous evaluation metrics
        anomalous_metrics = self.anomaly_generator.get_anomalous_metrics(accuracy, avg_loss)
        
        logger.info(f"[EVALUATION] Client {self.client_id} evaluation completed")
        logger.info(f"  True metrics: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")
        logger.info(f"  Reported metrics: Loss={anomalous_metrics['loss']:.4f}, Accuracy={anomalous_metrics['accuracy']:.4f}")
        
        return float(anomalous_metrics["loss"]), total, {
            "accuracy": float(anomalous_metrics["accuracy"]),
            "client_id": str(self.client_id),
            "poisoning_active": False,
            "anomaly_active": True,
            "anomaly_type": self.anomaly_generator.anomaly_type
        }

def start_anomalous_client(
    client_id=0, 
    server_address=None, 
    anomaly_type="mixed",
    anomaly_intensity=0.7,
    anomaly_frequency=0.8
):
    """Initialize and start an anomalous client."""
    
    if server_address:
        ANOMALY_CLIENT_CONFIG["server_address"] = server_address
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Data file
    data_path = f"framingham_part{client_id+1}.csv"
    if not os.path.exists(data_path):
        logger.error(f"Data file {data_path} not found")
        return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Anomaly configuration
    anomaly_config = {
        "anomaly_type": anomaly_type,
        "intensity": anomaly_intensity,
        "frequency": anomaly_frequency
    }
    
    # Create anomalous client
    client = AnomalousFraminghamClient(
        model, dataloader, device, client_id, anomaly_config
    )
    
    # Display client information
    print(f"\n{'='*80}")
    print(f"⚠️ ANOMALOUS FRAMINGHAM CLIENT {client_id}")
    print(f"{'='*80}")
    print(f"Server:           {ANOMALY_CLIENT_CONFIG['server_address']}")
    print(f"Data file:        {data_path}")
    print(f"Device:           {device}")
    print(f"")
    print(f"Anomaly Configuration:")
    print(f"  Type:           {anomaly_type}")
    print(f"  Intensity:      {anomaly_intensity} (0.0-1.0)")
    print(f"  Frequency:      {anomaly_frequency} (probability per round)")
    print(f"")
    print(f"Anomaly Types:")
    print(f"  parameter:      Parameter distribution anomalies")
    print(f"  training:       Training behavior anomalies")
    print(f"  performance:    Performance reporting anomalies")
    print(f"  statistical:    Statistical pattern anomalies")
    print(f"  mixed:          Combination of all types")
    print(f"")
    print(f"⚠️  This client will exhibit anomalous behavior for testing!")
    print(f"    (Not traditional model poisoning)")
    print(f"{'='*80}")
    print(f"\nConnecting to server...\n")
    
    # Start client
    fl.client.start_client(
        server_address=ANOMALY_CLIENT_CONFIG["server_address"], 
        client=client
    )

if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Configuration for Jupyter
        client_id = 5                    # Change this to match your setup
        anomaly_type = "mixed"           # "parameter", "training", "performance", "statistical", "mixed"
        anomaly_intensity = 0.9          # Increased to 0.9 for stronger detection
        anomaly_frequency = 1.0          # 100% frequency for testing
        
        print(f"⚠️ Starting ANOMALOUS client {client_id}")
        print(f"   Type: {anomaly_type}")
        print(f"   Intensity: {anomaly_intensity}")
        print(f"   Frequency: {anomaly_frequency}")
        
        start_anomalous_client(
            client_id=client_id,
            anomaly_type=anomaly_type,
            anomaly_intensity=anomaly_intensity,
            anomaly_frequency=anomaly_frequency
        )
    else:
        import argparse
        parser = argparse.ArgumentParser(description="Anomalous Framingham FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--type", type=str, default="mixed", 
                           choices=["parameter", "training", "performance", "statistical", "mixed"],
                           help="Type of anomaly to introduce")
        parser.add_argument("--intensity", type=float, default=0.7, help="Anomaly intensity (0.0-1.0)")
        parser.add_argument("--frequency", type=float, default=0.8, help="Anomaly frequency (0.0-1.0)")
        
        args = parser.parse_args()
        
        start_anomalous_client(
            client_id=args.id,
            server_address=args.server,
            anomaly_type=args.type,
            anomaly_intensity=args.intensity,
            anomaly_frequency=args.frequency
        )

2025-05-24 20:33:11,587 - AnomalousClient - INFO - Using device: cuda
2025-05-24 20:33:11,595 - AnomalousClient - INFO - Loaded framingham_part6.csv with shape (1060, 16)
2025-05-24 20:33:11,598 - AnomalousClient - INFO - Class distribution: {0: 898, 1: 162}
2025-05-24 20:33:11,602 - AnomalousClient - INFO - Created dataloader with 1060 samples and 15 features
2025-05-24 20:33:11,605 - AnomalousClient - INFO - Model initialized with input size: 15
2025-05-24 20:33:11,605 - AnomalousClient - INFO - [ANOMALOUS] Client 5 initialized
2025-05-24 20:33:11,606 - AnomalousClient - INFO -   Anomaly Type: mixed
2025-05-24 20:33:11,607 - AnomalousClient - INFO -   Intensity: 0.9
2025-05-24 20:33:11,608 - AnomalousClient - INFO -   Frequency: 1.0
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

  

Running in Jupyter/IPython environment
⚠️ Starting ANOMALOUS client 5
   Type: mixed
   Intensity: 0.9
   Frequency: 1.0

⚠️ ANOMALOUS FRAMINGHAM CLIENT 5
Server:           localhost:8080
Data file:        framingham_part6.csv
Device:           cuda

Anomaly Configuration:
  Type:           mixed
  Intensity:      0.9 (0.0-1.0)
  Frequency:      1.0 (probability per round)

Anomaly Types:
  parameter:      Parameter distribution anomalies
  training:       Training behavior anomalies
  performance:    Performance reporting anomalies
  statistical:    Statistical pattern anomalies
  mixed:          Combination of all types

⚠️  This client will exhibit anomalous behavior for testing!
    (Not traditional model poisoning)

Connecting to server...



INFO :      
2025-05-24 20:33:13,494 - flwr - INFO - 
INFO :      Received: train message f9f7297d-5f02-48cc-8617-51077596e3f4
2025-05-24 20:33:13,496 - flwr - INFO - Received: train message f9f7297d-5f02-48cc-8617-51077596e3f4
2025-05-24 20:33:13,498 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-24 20:33:13,502 - AnomalousClient - INFO - Client 5: parameters updated from server
2025-05-24 20:33:13,503 - AnomalousClient - INFO -   Using anomalously high learning rate: 0.09000000000000001
2025-05-24 20:33:13,505 - AnomalousClient - INFO - [ANOMALY] Client 5 using anomalous training config
2025-05-24 20:33:13,521 - AnomalousClient - INFO - Client 5 - Epoch 1/3 - Batch 0 - Loss: 0.6494
2025-05-24 20:33:13,622 - AnomalousClient - INFO - Client 5 - Epoch 1/3 - Batch 10 - Loss: 0.9142
2025-05-24 20:33:13,726 - An

In [16]:
#!/usr/bin/env python3
"""
Anomalous Federated Learning Client (Non-Poisoning)
===================================================

This client creates various types of anomalies without traditional model poisoning:
- Parameter distribution anomalies
- Training behavior anomalies  
- Performance inconsistencies
- Statistical outliers
- Pattern irregularities

These anomalies test the anomaly detection system specifically.
"""

import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import sys
import logging
import random
from typing import Dict, List, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("AnomalousClient")

# Client configuration
ANOMALY_CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01,
    
    # Anomaly configuration
    "anomaly_type": "mixed",  # "parameter", "training", "performance", "statistical", "mixed"
    "anomaly_intensity": 0.7,  # 0.0 (none) to 1.0 (maximum)
    "anomaly_frequency": 0.8,  # Probability of introducing anomaly each round
}

class HeartDiseaseModel(nn.Module):
    """Heart disease prediction model."""
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(ANOMALY_CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(ANOMALY_CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

class FedProxLoss(nn.Module):
    """FedProx loss function."""
    def __init__(self, base_criterion, mu=0.01):
        super(FedProxLoss, self).__init__()
        self.base_criterion = base_criterion
        self.mu = mu
        
    def forward(self, y_pred, y_true, model_params, global_params):
        base_loss = self.base_criterion(y_pred, y_true)
        
        proximal_term = 0.0
        if global_params is not None:
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
            loss = base_loss + (self.mu / 2) * proximal_term
            return loss
        
        return base_loss

class AnomalyGenerator:
    """Generates various types of anomalies for testing."""
    
    def __init__(self, anomaly_type="mixed", intensity=0.7, frequency=0.8):
        self.anomaly_type = anomaly_type
        self.intensity = intensity
        self.frequency = frequency
        self.round_count = 0
        
    def should_introduce_anomaly(self):
        """Decide whether to introduce anomaly this round."""
        return random.random() < self.frequency
    
    def apply_parameter_anomalies(self, parameters: List[np.ndarray]) -> List[np.ndarray]:
        """Apply parameter-level anomalies (non-poisoning)."""
        anomalous_params = []
        
        for i, param in enumerate(parameters):
            anomalous_param = param.copy()
            
            if self.anomaly_type in ["parameter", "mixed"]:
                # Type 1: Add structured noise (more aggressive)
                if random.random() < 0.7:
                    noise_scale = 0.3 * self.intensity  # Increased from 0.1
                    structured_noise = np.sin(np.arange(param.size) * 0.1).reshape(param.shape) * noise_scale
                    anomalous_param += structured_noise
                    logger.info(f"  Applied STRONG structured noise to layer {i} (scale={noise_scale:.3f})")
                
                # Type 2: Introduce more aggressive sparsity patterns
                if random.random() < 0.5:
                    sparsity_ratio = 0.4 * self.intensity  # Increased from 0.2
                    mask = np.random.random(param.shape) < sparsity_ratio
                    anomalous_param[mask] = 0
                    logger.info(f"  Applied AGGRESSIVE sparsity to layer {i} ({np.sum(mask)} zeros, {sparsity_ratio:.1%} sparse)")
                
                # Type 3: Amplify parameters more dramatically
                if random.random() < 0.6 and param.size > 10:
                    amplification = 1 + 1.5 * self.intensity  # Increased from 0.5
                    # Amplify every 2nd parameter instead of every 3rd
                    indices = np.arange(0, param.size, 2)
                    flat_param = anomalous_param.flatten()
                    flat_param[indices] *= amplification
                    anomalous_param = flat_param.reshape(param.shape)
                    logger.info(f"  Applied STRONG amplification to layer {i} (factor={amplification:.2f})")
                
                # Type 4: Add systematic bias (new)
                if random.random() < 0.4:
                    bias = 0.5 * self.intensity * np.sign(np.mean(param))
                    anomalous_param += bias
                    logger.info(f"  Added systematic bias to layer {i} (bias={bias:.3f})")
                
                # Type 5: Create artificial patterns (new)
                if random.random() < 0.3 and param.size > 50:
                    pattern_size = min(10, param.size // 10)
                    pattern = np.tile([1, -1], pattern_size // 2 + 1)[:pattern_size]
                    pattern = pattern * 0.3 * self.intensity
                    flat_param = anomalous_param.flatten()
                    flat_param[:pattern_size] = pattern
                    anomalous_param = flat_param.reshape(param.shape)
                    logger.info(f"  Applied artificial pattern to layer {i}")
            
            anomalous_params.append(anomalous_param)
        
        return anomalous_params
    
    def get_anomalous_training_config(self) -> Dict:
        """Get anomalous training configuration."""
        config = {}
        
        if self.anomaly_type in ["training", "mixed"]:
            # Anomalous learning rates
            if random.random() < 0.4:
                if random.random() < 0.5:
                    config["learning_rate"] = 0.1 * self.intensity  # Too high
                    logger.info(f"  Using anomalously high learning rate: {config['learning_rate']}")
                else:
                    config["learning_rate"] = 0.0001 * (1 - self.intensity)  # Too low
                    logger.info(f"  Using anomalously low learning rate: {config['learning_rate']}")
            
            # Anomalous batch processing
            if random.random() < 0.3:
                config["process_partial_batches"] = True
                config["partial_ratio"] = 0.3 + 0.4 * self.intensity
                logger.info(f"  Processing only {config['partial_ratio']:.2f} of each batch")
            
            # Anomalous epoch patterns
            if random.random() < 0.2:
                config["skip_epochs"] = int(1 + 2 * self.intensity)
                logger.info(f"  Skipping {config['skip_epochs']} training epochs")
        
        return config
    
    def get_anomalous_metrics(self, true_accuracy: float, true_loss: float) -> Dict:
        """Generate anomalous (but not obviously fake) metrics."""
        metrics = {
            "accuracy": true_accuracy,
            "loss": true_loss,
            "poisoning_active": False  # Not doing traditional poisoning
        }
        
        if self.anomaly_type in ["performance", "mixed"]:
            # Type 1: Extreme inconsistent performance (more aggressive)
            if random.random() < 0.6:
                # Very high accuracy but very high loss (clearly inconsistent)
                metrics["accuracy"] = min(0.95, true_accuracy + 0.4 * self.intensity)
                metrics["loss"] = max(true_loss, true_loss + 2.0 * self.intensity)
                logger.info(f"  Reporting EXTREME inconsistent metrics: acc={metrics['accuracy']:.3f}, loss={metrics['loss']:.3f}")
            
            # Type 2: Oscillating performance (more extreme)
            elif random.random() < 0.4:
                oscillation = 0.5 * self.intensity * np.sin(self.round_count * np.pi / 2)
                metrics["accuracy"] = max(0.05, min(0.95, true_accuracy + oscillation))
                metrics["loss"] = max(0.1, true_loss - oscillation * 0.5)
                logger.info(f"  Applying EXTREME performance oscillation: {oscillation:.3f}")
            
            # Type 3: Suspicious performance patterns
            elif random.random() < 0.3:
                # Performance that's too good for early rounds
                if self.round_count <= 3:
                    metrics["accuracy"] = 0.85 + 0.1 * self.intensity
                    metrics["loss"] = 0.2 + 0.1 * self.intensity
                    logger.info(f"  Reporting suspiciously good early performance")
                
                # Or performance that degrades suddenly
                else:
                    metrics["accuracy"] = max(0.1, true_accuracy - 0.3 * self.intensity)
                    metrics["loss"] = true_loss + 1.5 * self.intensity
                    logger.info(f"  Reporting sudden performance degradation")
        
        return metrics

def load_data(data_path):
    """Load and preprocess data."""
    try:
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        
        if df.isnull().sum().sum() > 0:
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing values: {df.shape}")
        
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' not found in dataset!")
            
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=ANOMALY_CLIENT_CONFIG["batch_size"], shuffle=True)
        
        logger.info(f"Created dataloader with {len(dataset)} samples and {X.shape[1]} features")
        return dataloader, X.shape[1]
    
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

class AnomalousFraminghamClient(fl.client.NumPyClient):
    """Anomalous client that generates various anomalies for testing."""
    
    def __init__(self, model, dataloader, device, client_id, anomaly_config=None):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.client_id = client_id
        self.global_params = None
        
        # Initialize anomaly generator
        if anomaly_config is None:
            anomaly_config = {
                "anomaly_type": ANOMALY_CLIENT_CONFIG["anomaly_type"],
                "intensity": ANOMALY_CLIENT_CONFIG["anomaly_intensity"],
                "frequency": ANOMALY_CLIENT_CONFIG["anomaly_frequency"]
            }
        
        self.anomaly_generator = AnomalyGenerator(**anomaly_config)
        
        logger.info(f"[ANOMALOUS] Client {client_id} initialized")
        logger.info(f"  Anomaly Type: {anomaly_config['anomaly_type']}")
        logger.info(f"  Intensity: {anomaly_config['intensity']}")
        logger.info(f"  Frequency: {anomaly_config['frequency']}")
        
    def get_parameters(self, config):
        """Get model parameters with potential anomalies."""
        params = [val.detach().cpu().numpy() for val in self.model.parameters()]
        
        # Apply anomalies if decided for this round
        if self.anomaly_generator.should_introduce_anomaly():
            logger.info(f"[ANOMALY] Client {self.client_id} introducing anomalies...")
            params = self.anomaly_generator.apply_parameter_anomalies(params)
            
            # Log parameter statistics
            original_norm = sum(np.linalg.norm(p) for p in [val.detach().cpu().numpy() for val in self.model.parameters()])
            anomalous_norm = sum(np.linalg.norm(p) for p in params)
            logger.info(f"  Parameter norm: {original_norm:.6f} -> {anomalous_norm:.6f}")
        else:
            logger.info(f"[NORMAL] Client {self.client_id} behaving normally this round")
        
        return params
    
    def set_parameters(self, parameters):
        """Set model parameters."""
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info(f"Client {self.client_id}: parameters updated from server")
    
    def fit(self, parameters, config):
        """Train with potential anomalies."""
        self.set_parameters(parameters)
        self.anomaly_generator.round_count += 1
        
        # Get anomalous training configuration
        anomaly_config = self.anomaly_generator.get_anomalous_training_config()
        
        # Apply anomalous training settings
        learning_rate = anomaly_config.get("learning_rate", ANOMALY_CLIENT_CONFIG["learning_rate"])
        process_partial = anomaly_config.get("process_partial_batches", False)
        partial_ratio = anomaly_config.get("partial_ratio", 1.0)
        skip_epochs = anomaly_config.get("skip_epochs", 0)
        
        if anomaly_config:
            logger.info(f"[ANOMALY] Client {self.client_id} using anomalous training config")
        
        self.model.train()
        criterion = nn.BCELoss()
        proximal_criterion = FedProxLoss(criterion, mu=ANOMALY_CLIENT_CONFIG["proximal_mu"])
        
        optimizer = torch.optim.Adam(
            self.model.parameters(), 
            lr=learning_rate,
            weight_decay=ANOMALY_CLIENT_CONFIG["weight_decay"]
        )
        
        total_loss = 0.0
        total_samples = 0
        correct = 0
        
        # Training with potential anomalies
        effective_epochs = max(1, ANOMALY_CLIENT_CONFIG["local_epochs"] - skip_epochs)
        
        for epoch in range(effective_epochs):
            epoch_loss = 0.0
            epoch_samples = 0
            
            for batch_idx, (X, y) in enumerate(self.dataloader):
                # Anomalous batch processing
                if process_partial and random.random() > partial_ratio:
                    continue  # Skip this batch
                
                X, y = X.to(self.device), y.to(self.device)
                
                # Forward pass
                y_pred = self.model(X)
                
                # Loss calculation
                loss = proximal_criterion(
                    y_pred, y, 
                    self.model.parameters(),
                    self.global_params
                )
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Update metrics
                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)
                
                # Calculate accuracy
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
                
                if batch_idx % 10 == 0:
                    logger.info(f"Client {self.client_id} - Epoch {epoch+1}/{effective_epochs} - Batch {batch_idx} - Loss: {loss.item():.4f}")
        
        # Calculate metrics
        avg_loss = total_loss / total_samples if total_samples > 0 else 0
        accuracy = correct / total_samples if total_samples > 0 else 0
        
        # Apply anomalous metric reporting
        anomalous_metrics = self.anomaly_generator.get_anomalous_metrics(accuracy, avg_loss)
        
        logger.info(f"[TRAINING] Client {self.client_id} training completed")
        logger.info(f"  True metrics: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")
        logger.info(f"  Reported metrics: Loss={anomalous_metrics['loss']:.4f}, Accuracy={anomalous_metrics['accuracy']:.4f}")
        
        return self.get_parameters({}), total_samples, {
            "loss": float(anomalous_metrics["loss"]), 
            "accuracy": float(anomalous_metrics["accuracy"]),
            "client_id": str(self.client_id),
            "poisoning_active": False,  # Not traditional poisoning
            "anomaly_active": True,     # But anomalous behavior
            "anomaly_type": self.anomaly_generator.anomaly_type,
            "anomaly_intensity": self.anomaly_generator.intensity
        }
    
    def evaluate(self, parameters, config):
        """Evaluate with potential anomalies."""
        self.set_parameters(parameters)
        
        self.model.eval()
        criterion = nn.BCELoss()
        
        loss = 0.0
        total = 0
        correct = 0
        
        with torch.no_grad():
            for X, y in self.dataloader:
                X, y = X.to(self.device), y.to(self.device)
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                
                loss += batch_loss * X.size(0)
                total += X.size(0)
                
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()
        
        avg_loss = loss / total if total > 0 else 0
        accuracy = correct / total if total > 0 else 0
        
        # Apply anomalous evaluation metrics
        anomalous_metrics = self.anomaly_generator.get_anomalous_metrics(accuracy, avg_loss)
        
        logger.info(f"[EVALUATION] Client {self.client_id} evaluation completed")
        logger.info(f"  True metrics: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")
        logger.info(f"  Reported metrics: Loss={anomalous_metrics['loss']:.4f}, Accuracy={anomalous_metrics['accuracy']:.4f}")
        
        return float(anomalous_metrics["loss"]), total, {
            "accuracy": float(anomalous_metrics["accuracy"]),
            "client_id": str(self.client_id),
            "poisoning_active": False,
            "anomaly_active": True,
            "anomaly_type": self.anomaly_generator.anomaly_type
        }

def start_anomalous_client(
    client_id=0, 
    server_address=None, 
    anomaly_type="mixed",
    anomaly_intensity=0.7,
    anomaly_frequency=0.8
):
    """Initialize and start an anomalous client."""
    
    if server_address:
        ANOMALY_CLIENT_CONFIG["server_address"] = server_address
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Data file
    data_path = f"framingham_part{client_id+1}.csv"
    if not os.path.exists(data_path):
        logger.error(f"Data file {data_path} not found")
        return
    
    # Load data
    dataloader, input_size = load_data(data_path)
    
    # Initialize model
    model = HeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size: {input_size}")
    
    # Anomaly configuration
    anomaly_config = {
        "anomaly_type": anomaly_type,
        "intensity": anomaly_intensity,
        "frequency": anomaly_frequency
    }
    
    # Create anomalous client
    client = AnomalousFraminghamClient(
        model, dataloader, device, client_id, anomaly_config
    )
    
    # Display client information
    print(f"\n{'='*80}")
    print(f"⚠️ ANOMALOUS FRAMINGHAM CLIENT {client_id}")
    print(f"{'='*80}")
    print(f"Server:           {ANOMALY_CLIENT_CONFIG['server_address']}")
    print(f"Data file:        {data_path}")
    print(f"Device:           {device}")
    print(f"")
    print(f"Anomaly Configuration:")
    print(f"  Type:           {anomaly_type}")
    print(f"  Intensity:      {anomaly_intensity} (0.0-1.0)")
    print(f"  Frequency:      {anomaly_frequency} (probability per round)")
    print(f"")
    print(f"Anomaly Types:")
    print(f"  parameter:      Parameter distribution anomalies")
    print(f"  training:       Training behavior anomalies")
    print(f"  performance:    Performance reporting anomalies")
    print(f"  statistical:    Statistical pattern anomalies")
    print(f"  mixed:          Combination of all types")
    print(f"")
    print(f"⚠️  This client will exhibit anomalous behavior for testing!")
    print(f"    (Not traditional model poisoning)")
    print(f"{'='*80}")
    print(f"\nConnecting to server...\n")
    
    # Start client
    fl.client.start_client(
        server_address=ANOMALY_CLIENT_CONFIG["server_address"], 
        client=client
    )

if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
        print("Running in Jupyter/IPython environment")
        # Configuration for Jupyter
        client_id = 5                    # Change this to match your setup
        anomaly_type = "mixed"           # "parameter", "training", "performance", "statistical", "mixed"
        anomaly_intensity = 0.9          # Increased to 0.9 for stronger detection
        anomaly_frequency = 1.0          # 100% frequency for testing
        
        print(f"⚠️ Starting ANOMALOUS client {client_id}")
        print(f"   Type: {anomaly_type}")
        print(f"   Intensity: {anomaly_intensity}")
        print(f"   Frequency: {anomaly_frequency}")
        
        start_anomalous_client(
            client_id=client_id,
            anomaly_type=anomaly_type,
            anomaly_intensity=anomaly_intensity,
            anomaly_frequency=anomaly_frequency
        )
    else:
        import argparse
        parser = argparse.ArgumentParser(description="Anomalous Framingham FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--type", type=str, default="mixed", 
                           choices=["parameter", "training", "performance", "statistical", "mixed"],
                           help="Type of anomaly to introduce")
        parser.add_argument("--intensity", type=float, default=0.7, help="Anomaly intensity (0.0-1.0)")
        parser.add_argument("--frequency", type=float, default=0.8, help="Anomaly frequency (0.0-1.0)")
        
        args = parser.parse_args()
        
        start_anomalous_client(
            client_id=args.id,
            server_address=args.server,
            anomaly_type=args.type,
            anomaly_intensity=args.intensity,
            anomaly_frequency=args.frequency
        )

2025-05-24 22:45:37,489 - AnomalousClient - INFO - Using device: cuda
2025-05-24 22:45:37,496 - AnomalousClient - INFO - Loaded framingham_part6.csv with shape (1060, 16)
2025-05-24 22:45:37,498 - AnomalousClient - INFO - Class distribution: {0: 898, 1: 162}
2025-05-24 22:45:37,504 - AnomalousClient - INFO - Created dataloader with 1060 samples and 15 features
2025-05-24 22:45:37,507 - AnomalousClient - INFO - Model initialized with input size: 15
2025-05-24 22:45:37,508 - AnomalousClient - INFO - [ANOMALOUS] Client 5 initialized
2025-05-24 22:45:37,509 - AnomalousClient - INFO -   Anomaly Type: mixed
2025-05-24 22:45:37,509 - AnomalousClient - INFO -   Intensity: 0.9
2025-05-24 22:45:37,510 - AnomalousClient - INFO -   Frequency: 1.0
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

  

Running in Jupyter/IPython environment
⚠️ Starting ANOMALOUS client 5
   Type: mixed
   Intensity: 0.9
   Frequency: 1.0

⚠️ ANOMALOUS FRAMINGHAM CLIENT 5
Server:           localhost:8080
Data file:        framingham_part6.csv
Device:           cuda

Anomaly Configuration:
  Type:           mixed
  Intensity:      0.9 (0.0-1.0)
  Frequency:      1.0 (probability per round)

Anomaly Types:
  parameter:      Parameter distribution anomalies
  training:       Training behavior anomalies
  performance:    Performance reporting anomalies
  statistical:    Statistical pattern anomalies
  mixed:          Combination of all types

⚠️  This client will exhibit anomalous behavior for testing!
    (Not traditional model poisoning)

Connecting to server...



INFO :      
2025-05-24 22:45:40,090 - flwr - INFO - 
INFO :      Received: train message 03388672-e75c-4b3e-815c-ac69abef5a8e
2025-05-24 22:45:40,093 - flwr - INFO - Received: train message 03388672-e75c-4b3e-815c-ac69abef5a8e
2025-05-24 22:45:40,093 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-24 22:45:40,111 - AnomalousClient - INFO - Client 5: parameters updated from server
2025-05-24 22:45:40,114 - AnomalousClient - INFO -   Using anomalously high learning rate: 0.09000000000000001
2025-05-24 22:45:40,116 - AnomalousClient - INFO -   Skipping 2 training epochs
2025-05-24 22:45:40,119 - AnomalousClient - INFO - [ANOMALY] Client 5 using anomalous training config
2025-05-24 22:45:40,136 - AnomalousClient - INFO - Client 5 - Epoch 1/1 - Batch 0 - Loss: 0.6825
2025-05-24 22:45:40,234 - AnomalousClient - IN